In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
import pandas as pd

In [ ]:
w = pd.read_csv("w.tsv", sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv("h.tsv", sep="\t", index_col=0)

h.columns.name = "Sample"

In [ ]:
import kraft

In [ ]:
gps_map = kraft.GPSMap(w=w.T, h=h)

In [ ]:
import os

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.plot(w_or_h)

In [ ]:
import numpy as np

In [ ]:
for w_or_h, elements, element_x_dimension in (
    ("w", gps_map.w_elements, gps_map.w_element_x_dimension),
    ("h", gps_map.h_elements, gps_map.h_element_x_dimension),
):

    annotation_x_element = pd.DataFrame(
        [
            np.apply_along_axis(np.sum, 1, element_x_dimension).astype(int),
            (np.apply_along_axis(np.sum, 1, element_x_dimension) * 10).astype(int),
            np.apply_along_axis(np.sum, 1, element_x_dimension),
        ],
        index=("Binary", "Categorical", "Continuous"),
        columns=elements,
    )

    annotation_x_element.iloc[0, 0] = np.nan

    annotation_x_element.iloc[1, 1] = np.nan

    annotation_x_element.iloc[2, 2] = np.nan

    annotation_x_element.iloc[:, 3] = np.nan

    #     for (
    #         _,
    #         element_value,
    #     ) in annotation_x_element.iterrows():

    #         gps_map.plot(
    #             w_or_h,
    #             annotation_x_element=element_value.to_frame().T,
    #         )

    gps_map.plot(w_or_h, annotation_x_element=annotation_x_element)

    gps_map.plot(
        w_or_h,
        annotation_x_element=annotation_x_element.iloc[[-1]],
        annotation_std_maxs=(1,),
    )

    gps_map.plot(
        w_or_h,
        annotation_x_element=annotation_x_element.iloc[[-1]],
        annotation_std_maxs=(10,),
    )

In [ ]:
for w_or_h, element_label in (
    ("w", w.idxmax(axis=1).str[1:].astype(int)),
    ("h", h.idxmax().str[1:].astype(int)),
):

    gps_map.set_element_label(w_or_h, element_label)

    gps_map.plot(w_or_h)

In [ ]:
for w_or_h, w_or_h_matrix in (("w", w.T), ("h", h)):

    print(gps_map.predict(w_or_h, w_or_h_matrix.iloc[:, :8]))

In [ ]:
for w_or_h in ("w", "h"):

    gps_map.anneal(w_or_h, n_iteration=64, initial_temperature=0.01)

    gps_map.plot(w_or_h)

In [ ]:
h = pd.read_csv("kras_gps_map.h.tsv", sep="\t", index_col=0)

h = h.apply(lambda row: (row - row.mean()) / row.std(), axis=1)

h = h.clip(lower=-3, upper=3)

h = h.apply(lambda row: (row - row.min()) / (row.max() - row.min()), axis=1)

gps_map = kraft.GPSMap(h=h, h_pull_power=2.4, plot=False)

In [ ]:
h_element_label = pd.read_csv(
    "kras_gps_map.k_x_h_element.tsv", sep="\t", index_col=0
).loc["K15"]

gps_map.set_element_label(
    "h",
    h_element_label,
    label_colors=(
        "#e74c3c",
        "#ffd700",
        "#4b0082",
        "#993300",
        "#4169e1",
        "#90ee90",
        "#f4bd60",
        "#8b008b",
        "#fa8072",
        "#b0e0e6",
        "#20d9ba",
        "#da70d6",
        "#d2691e",
        "#dc143c",
        "#2e8b57",
    ),
)

gps_map.plot("h")

In [ ]:
gps_map.predict("h", h)

In [ ]:
gps_map_path = os.path.join(kraft.normalize_path("~"), "gps_map.pickle.gz")

kraft.write_gps_map(gps_map, gps_map_path)

In [ ]:
gps_map = kraft.read_gps_map(gps_map_path)

gps_map.plot("h")